# Курсовая работа

Подключим неоходимое для работы:

In [5]:
import math
from math import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import filterpy
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_continuous_white_noise
from filterpy.common import Q_discrete_white_noise
#import code.book_plots as bp
from scipy import stats
from filterpy.common import van_loan_discretization
from scipy.linalg import expm
%matplotlib inline

ImportError: No module named filterpy

Переведём данные в удобный для работы формат и далее сохраним их в пикл (чтобы можно было быстро работать с этой структурой данных):

In [3]:
def CNFparser(filename):
    '''
    Тут мы расписываем парсер
    CNFnav - кусок для работы в режиме навигации
    CNFalign - кусок для выставки (причём секунд 10-20 из него для алгебраической, остальное на кинематику)
    '''
    global CNFnav
    global CNFalign
    CNFnames = ['Time','AccX','AccY','AccZ','OmX','OmY','OmZ']
    CNF = pd.read_table(filename, sep = '\s+' ,header = None, names = CNFnames)
    #CNFalign = CNF[CNF.FlagAlignment == 1]
    #CNFalign = CNFalign.loc[:,['Time','OmX','OmY','OmZ','AccX','AccY','AccZ']]
    #CNFnav = CNF[CNF.FlagAlignment == 0]
    #CNFnav = CNFnav.loc[:,['Time','OmX','OmY','OmZ','AccX','AccY','AccZ']]
    #CNF = []
    CNF['Invariant'] = CNFalign.AccX*CNFalign.OmX +  CNFalign.AccY*CNFalign.OmY +  CNFalign.AccZ*CNFalign.OmZ
    CNF['g\''] = (CNFalign.AccX**2 + CNFalign.AccY**2 + CNFalign.AccZ**2)**(0.5)
    CNF['W_Std500'] = (CNFalign.Invariant.rolling(window = 500, center = False).std()).dropna()

In [4]:
CNFparser('IMU_coursework.txt')

NameError: global name 'pd' is not defined

Определим функции для определения параметров:

In [5]:
def g_phi(phi, h, a, e2):
    g = 9.78030 * (1 - 2 * h / a + 3 * e2 * sin(phi) ** 2 / 4);
    return g